# Assignment 1 - Evolutionary Computation
To reproduce our results, you can simply run all cells in this notebook. A seed was added for reproduceability, but this was done after some experimental results were already added to our assignment pdf, so results might vary slightly for assignments 6 and 8, also due to stochasticity. 
Please make sure to install the deap library, by running the following cell:

In [ ]:
# !pip install deap

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import operator
import math
import random
from deap import base, creator, gp, tools, algorithms

## Seed

In [ ]:
np.random.seed(0)

## Load Data

In [ ]:
tsp_problem = np.loadtxt("file-tsp.txt")

## Exercise 4 - Role of Selection in GA's

In [ ]:
def inverse(x):
    return 1 if x == 0 else 0

In [ ]:
def counting_ones(l, p, iterations):
    x = np.random.choice([0, 1], size=(l))
    fitness = []

    iteration = 0
    while((iteration < iterations) & (np.sum(x) < l)):
        x_m = [inverse(x_i) if (np.random.random() < p) else x_i for x_i in x]
        
        if((l - np.sum(x_m)) < (l - np.sum(x))):
            x = x_m
            
        fitness.append((l - np.sum(x)))
        iteration += 1
            
    return fitness

In [ ]:
def plot_fitness(ax, fitness, iterations):
    ax.plot(fitness)
    ax.set_ylim(0, np.max(fitness))
    ax.set_xlim(0, iterations)
    ax.set_xticks(np.arange(0, iterations+1, 300))
    ax.set_xlabel("Iterations")
    ax.set_ylabel("Fitness")

In [ ]:
l = 100 
p = 1/l
iterations = 1500

fitness = counting_ones(l, p, iterations)
fig, axs = plt.subplots()
plot_fitness(axs, fitness, iterations)

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(20, 8))
axs = axs.ravel()

for ax in axs:
    fitness = counting_ones(l, p, iterations)
    plot_fitness(ax, fitness, iterations)

In [ ]:
# This count ones function is the modified version

def counting_ones(l, p, iterations):
    x = np.random.choice([0, 1], size=(l))
    fitness = []

    iteration = 0
    while((iteration < iterations) & (np.sum(x) < l)):
        x_m = [inverse(x_i) if (np.random.random() < p) else x_i for x_i in x]
        x = x_m
            
        fitness.append((l - np.sum(x)))
        iteration += 1
            
    return fitness

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(20, 8))
axs = axs.ravel()

for ax in axs:
    fitness = counting_ones(l, p, iterations)
    plot_fitness(ax, fitness, iterations)

## Exercise 6 - Mementic Algorithms vs Simple EAs

In [ ]:
plt.scatter(tsp_problem[:,0],tsp_problem[:,1])
plt.show()
print(tsp_problem[:3])
print(len(tsp_problem))

In [ ]:
def initialise_population(n_individuals, individual_len):
    population = np.zeros((n_individuals, individual_len))
    
    for i in range(n_individuals):
        x = np.arange(individual_len)
        random.shuffle(x)
        population[i,:] = x
        
    return population.astype(int)

In [ ]:
def get_distance(individual):
    distance = 0
    for i in range(individual.size-1):
        x = tsp_problem[individual[i],0] - tsp_problem[individual[i+1],0]
        y = tsp_problem[individual[i],1] - tsp_problem[individual[i+1],1]
        
        d = np.sqrt((x**2)+(y**2))
        distance += d
        
    return distance

In [ ]:
def opt_2(individual,i,k):
    x = individual[i]
    y = individual[k]
    
    individual[i]=y
    individual[k]=x
    
    return individual

In [ ]:
def local_search(individual):
    best_individual = individual
    best_distance = get_distance(best_individual)
    converged = False
    
    while(converged != True):
        improvement = False
        
        for i in range(len(best_individual)-1):
            new_individual = opt_2(best_individual,i,i+1)
            new_distance = get_distance(new_individual)
            
            if new_distance<best_distance:
                best_individual = new_individual
                best_distance = new_distance
                improvement = True
                
        if improvement == False:
            converged = True
            
    return best_individual.astype(int)

In [ ]:
def evaluate(individual):
    return 1/get_distance(individual)

In [ ]:
def binary_tournament(population):
    population_index = np.arange(len(population))
    option_1 = random.choice(population_index)
    option_2 = random.choice(population_index)
    val_1 = evaluate(population[option_1,:])
    val_2 = evaluate(population[option_2,:])
    
    if val_1 > val_2:
        return population[option_1].astype(int)
    else:
        return population[option_2].astype(int)

In [ ]:
def crossover(parent_1, parent_2):
    indices = np.arange(len(parent_1))
    pick_1 = random.choice(indices)
    pick_2= random.choice(indices)
    if pick_1>pick_2:
        cut_1 = pick_2
        cut_2 = pick_1
    else:
        cut_1 = pick_1
        cut_2 = pick_2
        
    crossover_part_1 = parent_1[cut_1:cut_2]
    crossover_part_2 = parent_2[cut_1:cut_2]
    
    child_1 = np.zeros(len(parent_1))
    child_2 = np.zeros(len(parent_2))
    
    child_1[cut_1:cut_2] = crossover_part_2
    child_2[cut_1:cut_2] = crossover_part_1
    
    checklist_1 = np.concatenate([parent_1[cut_2:],parent_1[:cut_2]])
    checklist_2 = np.concatenate([parent_2[cut_2:],parent_2[:cut_2]])
    
    checklist_1 = [x for x in checklist_1 if x not in crossover_part_2]
    checklist_2 = [x for x in checklist_2 if x not in crossover_part_1]
    
    loop_number = len(checklist_1)
    
    for i in range(loop_number):
        child_1[(cut_2+i) % (len(child_1))] = checklist_1[0]
        child_2[(cut_2+i) % (len(child_2))] = checklist_2[0]
        del checklist_1[0]
        del checklist_2[0]
       
    return child_1.astype(int), child_2.astype(int)

In [ ]:
def mutate(individual,probability):
    threshold = random.random()
    if threshold<=probability:
        pick_1 = random.choice(individual)
        index_1 = np.where(individual == pick_1)
        pick_2 = random.choice(individual)
        index_2 = np.where(individual == pick_2)
        
        individual[index_1] = pick_2
        individual[index_2] = pick_1
        
        return individual.astype(int)
    else:
        return individual.astype(int)

In [ ]:
# Parameters
population_size = 20
n_offspring = 20
mutate_prob = 0.005
iterations = 1500

def memetic_algorithm(population_size,n_offspring,mutate_prob,iterations):
    # Initalize population
    population = initialise_population(population_size,len(tsp_problem))
    fitness = np.zeros(len(population))
    fitness_array = np.zeros((iterations,3))

    # Local Search
    for i in range(len(population)):
        population[i,:] = local_search(population[i,:])
        # Evaluate
        fitness[i] = evaluate(population[i,:])

    # Loop over iterations
    for n in range(iterations):
        
        # Binary tournament selection
        parents = np.zeros((n_offspring,len(tsp_problem))).astype(int)
        for i in range(0, n_offspring, 2):
            parents[i] = binary_tournament(population)
            parents[i+1] = binary_tournament(population)

        children = np.zeros((n_offspring,len(tsp_problem))).astype(int)
        
        # Crossover
        for i in range(0, n_offspring, 2):
            children[i],children[i+1] = crossover(parents[i],parents[i+1])

        # Mutate
        for i in range(len(children)):
            children[i,:] = mutate(children[i],mutate_prob)

        # Local Search
        for i in range(len(children)):
            children[i,:] = local_search(children[i,:])

        population = children
        
        # Evaluate
        for i in range(len(population)):
            fitness[i] = evaluate(population[i,:])

        fitness_array[n,0] = np.min(fitness)
        fitness_array[n,1] = np.max(fitness)
        fitness_array[n,2] = np.average(fitness)
        
        if n%500 == 0:
            print("Iteration:",n)
            
    return fitness_array

In [ ]:
# Parameters
MA_fitness = np.zeros((10,1500,3))
population_size = 20
n_offspring = 20
mutate_prob = 0.005
iterations = 1500

fig, axs = plt.subplots(2,5, figsize=(20,8))
axs = axs.ravel()

for i,ax in enumerate(axs):
    MA_fitness[i,:,:] = memetic_algorithm(population_size,n_offspring,mutate_prob,iterations)
    ax.plot(MA_fitness[i,:,2])
    ax.plot(MA_fitness[i,:,0])
    ax.set_title("Memetic Algorithm")
    ax.legend(["Average","Best"])

In [ ]:
# Parameters
population_size = 20
n_offspring = 20
mutate_prob = 0.01
iterations = 1500

def evolutionary_algorithm(population_size,n_offspring,mutate_prob,iterations):
    # Initialize population
    population = initialise_population(population_size,len(tsp_problem))
    fitness = np.zeros(len(population))
    fitness_array_ea = np.zeros((iterations,3))
    
    # Loop over iterations
    for n in range(iterations):
        parents = np.zeros((n_offspring,len(tsp_problem))).astype(int)
        for i in range(0, n_offspring, 2):
            parents[i] = binary_tournament(population)
            parents[i+1] = binary_tournament(population)

        children = np.zeros((n_offspring,len(tsp_problem))).astype(int)
        
        # Crossover 
        for i in range(0, n_offspring, 2):
            children[i],children[i+1] = crossover(parents[i],parents[i+1])

        # Mutate
        for i in range(len(children)):
            children[i,:] = mutate(children[i],mutate_prob)

        population = children
        
        # Evaluation
        for i in range(len(population)):
            fitness[i] = evaluate(population[i,:])

        fitness_array_ea[n,0] = np.min(fitness)
        fitness_array_ea[n,1] = np.max(fitness)
        fitness_array_ea[n,2] = np.average(fitness)
        
        if n%500 == 0:
            print("Iteration:",n)
            
    return fitness_array_ea

In [ ]:
# Parameters
EA_fitness = np.zeros((10,1500,3))
population_size = 20
n_offspring = 20
mutate_prob = 0.005
iterations = 1500

fig, axs = plt.subplots(2,5, figsize=(20,8))
axs = axs.ravel()

for i,ax in enumerate(axs):
    EA_fitness[i,:,:] = evolutionary_algorithm(population_size,n_offspring,mutate_prob,iterations)
    ax.plot(EA_fitness[i,:,2])
    ax.plot(EA_fitness[i,:,0])
    ax.set_title("Evolutionary Algorithm")
    ax.legend(["Average","Best"])

## Exercise 8 - Genetic Programming Behaviour

In [ ]:
# The data we want to fit on
data = [[-1.0, 0.0000],
        [-0.9, -0.1629],
        [-0.8, -0.2624],
        [-0.7, -0.3129],
        [-0.6, -0.3264],
        [-0.5, -0.3125],
        [-0.4, -0.2784],
        [-0.3, -0.2289],
        [-0.2, -0.1664],
        [-0.1, -0.0909],
        [0, 0.0],
        [0.1, 0.1111],
        [0.2, 0.2496],
        [0.3, 0.4251],
        [0.4, 0.6496],
        [0.5, 0.9375],
        [0.6, 1.3056],
        [0.7, 1.7731],
        [0.8, 2.3616],
        [0.9, 3.0951],
        [1.0, 4.0000]]

In [ ]:
# Implemented a GP program based on this tutorial: https://deap.readthedocs.io/en/master/examples/gp_symbreg.html
pset = gp.PrimitiveSet("main", 1)
pset.renameArguments(ARG0="x")

In [ ]:
# Primitives
def div(x, y):
    return x/y if (y > 0) else 0

def log(x):
    return math.log(x) if (x > 0) else 0

def exp(x):
    return math.exp(min(x,100))

pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(div, 2)
pset.addPrimitive(log, 1)
pset.addPrimitive(exp, 1)
pset.addPrimitive(math.sin, 1)
pset.addPrimitive(math.cos, 1)

In [ ]:
# We want to maximize the fitness function
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [ ]:
# Fitness function
def absolute_error_sum(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    
    # Sum of absolute errors
    error = np.sum([abs(func(x[0]) - x[1]) for x in points])    

    # Return -sum of absolute errors
    return -error,

In [ ]:
### Toolbox
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", absolute_error_sum, points=data)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [ ]:
# Statistics
def min_(x):
    if type(x[0]) is list:
        x = np.asarray(x)
        return np.min(x[:,0])
    else:
        return np.min(x)
    
def max_(x):
    if type(x[0]) is list:
        x = np.asarray(x)
        return np.max(x[:,0])
    else:
        return np.max(x)
    
def avg_(x):
    if type(x[0]) is list:
        x = np.asarray(x)
        return np.mean(x[:,0])
    else:
        return np.mean(x)
    
def std_(x):
    if type(x[0]) is list:
        x = np.asarray(x)
        return np.std(x[:,0])
    else:
        return np.std(x)
    
def best_(x):
    if type(x[0]) is list:
        x = np.asarray(x)
        return x[np.argmax(x[:,1])][0]
    else:
        return np.max(x)
    
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(lambda ind: [len(ind), ind.fitness.values[0]])
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", avg_)
mstats.register("std", std_)
mstats.register("min", min_)
mstats.register("max", max_)
mstats.register("best", best_)

In [ ]:
# Evolution
pop = toolbox.population(n=1000)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0, ngen=50, stats=mstats,
                               halloffame=hof, verbose=True)

In [ ]:
# Plot fitness of best individual of each generation
best_fitness = log.chapters["fitness"].select("best")
plt.plot(best_fitness)
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.show()

In [ ]:
# Plot size 
min_size = log.chapters["size"].select("min")
max_size = log.chapters["size"].select("max")
avg_size = log.chapters["size"].select("avg")
best_size = log.chapters["size"].select("best")

In [ ]:
plt.plot(avg_size, label="Average Size")
plt.plot(best_size, label="Best Size")
plt.fill_between(np.arange(0,51), min_size, max_size, color="gainsboro", label="Spread")
plt.xlabel("Generation")
plt.ylabel("Tree Size")
plt.legend()
plt.show()